In [1]:
import os
from tqdm import tqdm
from absl import flags, app

import jax
import jax.numpy as jnp
from jax.example_libraries import optimizers as jax_opt

from numpy import Inf, NaN
import numpy as np

from torch.utils.tensorboard import SummaryWriter

from utils.rnn_utils import update
from utils.rnn_utils import rnn_params
from utils.rnn_utils import initialize_wout
from utils.rnn_utils import forward_rnn_interp

from utils.utils import setup_logging_directory
from utils.utils import visualize_mocap_interpolation

from utils.mocap_utils import get_mocap_data

/home/mirko/anaconda3/lib/python3.9/site-packages/jax/_src/api_util.py:172: SyntaxWarning: Jitted function has static_argnums=(4, 5), but only accepts 5 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "
/home/mirko/anaconda3/lib/python3.9/site-packages/jax/_src/api_util.py:172: SyntaxWarning: Jitted function has static_argnums=(5,), but only accepts 5 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "
/home/mirko/anaconda3/lib/python3.9/site-packages/jax/_src/api_util.py:172: SyntaxWarning: Jitted function has static_argnums=(4, 5, 6, 7, 8, 9), but only accepts 9 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "


In [2]:
folder = "motion_data_numpy/"
dataset_names = ["walk_15", "run_55"]
_,_,mean,std = get_mocap_data(folder,dataset_names=dataset_names)

In [19]:
folder = './logs/mocap_conceptor_loss_6/'
ckpt_path = os.path.join(folder, "ckpt")
epoch = 251
params_file = f'params_{epoch}.npz'
conceptor_file = f'conceptor_{epoch}.npz'

params = np.load(os.path.join(ckpt_path,params_file), allow_pickle=True)
params = dict(params)
rnn_size = params['w'].shape[0]
conceptors = np.load(os.path.join(ckpt_path,conceptor_file), allow_pickle=True)
conceptors = dict(conceptors)
c_matrix = [conceptors[f'C_{i+1}'] for i in range(len(conceptors))]

len_seqs = 400
lamdas = [0, 0.25, 0.5, 0.75, 1]

states = []
y_interp = []
for lamda in lamdas:
    t_interp = jnp.ones(len_seqs) * lamda
    ut_interp = jnp.zeros(len_seqs)
    yx_interpolation = forward_rnn_interp(
        params, c_matrix, ut_interp, None, t_interp=t_interp)

    x_interpolation = yx_interpolation[:, -rnn_size:]
    y_interpolation = yx_interpolation[:, :-rnn_size]
    states.append(x_interpolation)
    y_interp.append(y_interpolation)

In [20]:
from video_processing import convert_motion_data_to_video
from skeleton import *
import base64
from IPython.display import HTML

# load initial conditions
data_ini_walk = np.load("motion_data_numpy/data_ini_walk.npy")
data_ini_walk = [[data_ini_walk[0], data_ini_walk[1]], [data_ini_walk[2]]]

c_joints = np.load('motion_data_numpy/c_joint.npy', allow_pickle=True).item()
norm_ij = np.load('motion_data_numpy/norm_ij.npy')

files = convert_motion_data_to_video(folder, np.array(y_interp), mean,std, [f"epoch_{epoch}_lambda_{l}" for l in lamdas])

In [21]:
from IPython.display import HTML

width = 320
height = 240

html = ""
for file in files:
    html += """
    <h2>{2}</h2>
    <video width="{0}" height="{1}" controls>
      <source src="{2}" type="video/mp4">
    </video>
    """.format(width, height, file)

display(HTML(html))